<a href="https://colab.research.google.com/github/Yanbing-Judy/190DD-Project/blob/main/Project_Part1a_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CMPCS 190DD/ PSTAT 197A Project Part 1

Author: Lu Han, Judy Zhu

(a): Initial $S_{v,c}^{(0)}$ probability mass distribution (PMF) as well as initial infected population sizes $\textit{$I_{c}$}$, and model parameters $\textit{$L_{c}$}$ and γ for a single leaf node are given in $\textbf{part1a.npz}$. Choose arbitrary $\beta_{v,c}$ values between in [0,1] such that $\beta_{v,c}$ are non decreasing as v or c increases. In other words, $\beta_{v,0}$ $\leq$ $\beta_{v,1}$ $\leq$ $\beta_{v,2}$ $\leq$ $\beta_{v,3}$ for all v and $\beta_{0,c}$ $\leq$ $\beta_{1,c}$ $\leq$ $\beta_{2,c}$ $\leq$ $\beta_{3,c}$ for all c.

In [ ]:
import numpy as np
import pandas as pd

from scipy import integrate
from scipy.integrate import solve_ivp

from itertools import product
import matplotlib.pyplot as plt
import matplotlib as mpl

import seaborn as sns

In [52]:
part1a = np.load('part1a.npz')

print(part1a.files)

for i in part1a:
    print(i)
    print(part1a[i])
    
N = part1a['N']
Svc_0_pmf = part1a['Svc_0_pmf']
Lc = part1a['Lc']
Ic_0 = part1a['Ic_0']
gamma = part1a['gamma']
# initial conditions
Svc = np.multiply(SVC_0_pmf, N - np.sum(Ic_0)).flatten()
y0 = np.concatenate((Svc, Ic_0, R)) 
print(Svc.sum(axis=0).sum(axis=0), Ic_0, np.zeros(1))

['N', 'Svc_0_pmf', 'Lc', 'Ic_0', 'gamma']
N
100000.0
Svc_0_pmf
[[0.10953309 0.03300776 0.08179044 0.11593994]
 [0.03089752 0.08511994 0.04528764 0.05718283]
 [0.07471465 0.02495643 0.10236599 0.04256696]
 [0.01605863 0.08109608 0.09158212 0.00789999]]
Lc
[0.3 0.4 0.7 1. ]
Ic_0
[47 90 51 65]
gamma
0.07142857142857142
99746.99999999997 [47 90 51 65] [0.]


1. Using the model dynamics described in the previous section, and all the given and chosen parameters, simulate the behavior of the disease for 120 days for this single leaf node.

In [53]:
#SIR model
def SIR_model(t, Y, beta, gamma):
    #define RHS
    S = Y[:16]
    I = Y[16:20]
    R = Y[20]
    
    dS_dt = -(beta*S*I.sum()/N)
    dI_dt = -dS_dt.reshape(4,4).sum(axis=0)-gamma*I
    dR_dt = gamma*I.sum()
    
    RHS = np.concatenate((dS_dt, dI_dt, [dR_dt]), axis = 0)
    
    return RHS 

In [54]:
np.random.seed(190)
b = np.random.rand(16).reshape(4,4)
b = np.sort(b, axis = 0)
b = np.sort(b, axis = 1)
b = np.array(b.flatten())


sol = solve_ivp(fun=SIR_model, t_span=(0,119,1), y0=y0,\
                dense_output=True, args=[b,gamma])


2. Plot $S_{v,c}$ and $I_{c}$ values over time. (You can plot all $S_{v,c}$ on the same plot. Same for $\textit{$I_{c}$}$.). Also plot the overall $\textit{S, I, R}$ and $\textit{L}$ values.

In [ ]:
# Svc_over_time
plt.figure()
for i in range (16):
    Svc_over_time=sol.y[i]
    plt.plot(t, Svc_over_time, label="S(t)")
plt.xlabel("t (days)")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Ic_over_time
for i in range (16,20):
    Ic_over_time=sol.y[i]
    sns.oineplot(sol.t,)


In [ ]:
# S_overall, I_overall, R_overall, L_overall

S_overall=sol.y[0]
I_overall=sol.y[1]
R_overall=sol.y[2]
L_overall=Lc*I_overall

plt.figure()
plt.plot(t, S_over_time, label="S(t)")
plt.plot(t, I_over_time, label="I(t)")
plt.plot(t, R_over_time, label="R(t)")
plt.xlabel("t (days)")
plt.legend()
    
mpl.style.use('default')
plt.figure(figsize=(10,5))
p0=plt.plot(t, Lc*soln_obj.y[1], linewidth=2, label="MMSE estimated L(t)")
plt.plot(t, observed_data, 'o', label="Observed L(t)", color=p0[0].get_color(), markersize=5)
plt.xlabel("t")
plt.legend()
plt.tight_layout()
plt.show()

plt.show()

3. Do the shapes of $\textit{S, I, R}$ look similar to what you expected? Which of these plots is the “curve” people refer to when they say “flatten the curve”?


Yes, these shapes of S,I,R look similar to what I expected. ...

4. Do you observe that some $S_{v,c}$ compartments converged to zero while others converged to a positive value? Why do you think that is?

Yes. ...

5. Print the percentages of population that never got infected for all compartments (i.e. all values of v and c).

In [ ]:


print("The percentage of population that never got infected for all compartments is ", percentage)

6. Multiply all the $\beta_{v,c}$ values by 1/4. What happened to the $\textit{S, I, R}$ plots? Did the “curve” flatten compared to the previous case? Print the percentages of population that never got infected with these βv,c values.